In [1]:
!pip install catboost
# if you want to run in collab, uncomment the line below and make sure to cd/ls into the Boxing_punches_classification
# and run!
# !git clone https://github.com/MarwanBit/Boxing_punches_classification


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\Marwa\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
%cd Boxing_punches_classification

[WinError 2] The system cannot find the file specified: 'Boxing_punches_classification'
c:\Users\Marwa\OneDrive\Documents\GitHub\P-Boxing\Boxing_punches_classification


'ls' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn

import plotly.graph_objs as go
import plotly.express as px

from scipy.stats import moment, skew, kurtosis


from joblib import load
import os

import os
import sys


from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import roc_auc_score, f1_score, confusion_matrix, recall_score, precision_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

from catboost import CatBoostClassifier

from joblib import dump, load

In [5]:
'''
This cell will concern itself with declaring all of the functionality of the
data processing steps, alot of the functions for data processing
'''
def skip_fun(x):
    return x in (0, 1, 2)

cwd = os.getcwd()
print(cwd)

c:\Users\Marwa\OneDrive\Documents\GitHub\P-Boxing\Boxing_punches_classification


In [6]:
'''
Now we will create a segment which will load up all the files for a given folder, for this example we will consier train_M
'''
train_M_path = "data/train_file/train_M/"
train_d1_path = "data/train_file/train_d1/"
train_B_path = "data/train_file/train_B/"

train_M = os.listdir(train_M_path)
train_B = os.listdir(train_B_path)
train_d1 = os.listdir(train_d1_path)
#Now we will iterate through the folders
#Now let's load this file
first_file_path = train_M_path + train_M[0]
#Now let's try loading this first file


In [7]:
#Now let's try writing a file which takes in the first concatenates all the files together
train_M_dataset = pd.DataFrame()
for file_name in train_M:
    print(file_name)
file_index = 0
for file_name in train_M:
    data_i = pd.read_csv(train_M_path + file_name, skiprows = skip_fun)
    data_i["file_ind"] = file_index
    train_M_dataset = pd.concat([train_M_dataset, data_i], ignore_index = True)
    file_index += 1
train_M_dataset.rename(columns={"Time (s)": "Time",
                     " X (m/s2)": "X",
                     " Y (m/s2)": "Y",
                     " Z (m/s2)": "Z",
                     " R (m/s2)": "R",
                     " Theta (deg)": "Theta",
                     " Phi (deg)": "Phi",}, inplace=True)

train_M_dataset.sort_values(by=["file_ind", "Time"], inplace=True)


#Now we have some combined train_M_dataset
train_M_dataset


000_Accelerometer Data 2022-10-21 15-53-25.txt
001_Accelerometer Data 2022-10-21 16-09-06.txt
002_Accelerometer Data 2022-10-21 15-51-57.txt
003_Accelerometer Data 2022-10-21 15-48-31.txt
004_Accelerometer Data 2022-10-21 15-55-43.txt
005_Accelerometer Data 2022-10-21 16-24-09.txt
006_Accelerometer Data 2022-10-21 16-19-36.txt
007_Accelerometer Data 2022-10-21 16-04-56.txt
008_Accelerometer Data 2022-10-21 16-03-25.txt
009_Accelerometer Data 2022-10-21 16-07-54.txt
010_Accelerometer Data 2022-10-21 16-21-09.txt
011_Accelerometer Data 2022-10-21 16-23-09.txt
012_Accelerometer Data 2022-10-21 18-15-08.txt
013_Accelerometer Data 2022-10-21 18-30-58.txt
014_Accelerometer Data 2022-10-21 18-30-14.txt
015_Accelerometer Data 2022-10-21 18-29-09.txt
016_Accelerometer Data 2022-10-21 18-20-21.txt
017_Accelerometer Data 2022-10-21 18-16-27.txt
018_Accelerometer Data 2022-10-21 18-27-26.txt
019_Accelerometer Data 2022-10-21 18-24-08.txt
020_Accelerometer Data 2022-10-21 18-13-48.txt
021_Accelerom

,Time,X,Y,Z,R,Theta,Phi,file_ind
0,0.000000,2.626666,9.605117,-0.391113,9.965471,92.249260,74.705528,0
1,0.002492,2.629061,9.648235,-0.525262,10.013806,93.006760,74.757462,0
2,0.004984,2.607502,9.645840,-0.642641,10.012706,93.679916,74.873123,0
3,0.007475,2.571569,9.676982,-0.731274,10.039508,94.177109,75.118118,0
4,0.009967,2.571569,9.655422,-0.798348,10.023847,94.568153,75.086365,0
...,...,...,...,...,...,...,...,...
416015,24.509064,0.135346,1.220866,9.370549,9.450716,7.468106,83.673988,24
416016,24.511562,0.197629,1.299918,9.320244,9.412534,8.030035,81.355408,24
416017,24.514059,0.209606,1.333455,9.214842,9.313181,8.333642,81.066742,24
416018,24.516556,0.233562,1.350224,9.111835,9.214293,8.552305,80.186096,24


In [8]:
#Now let's perform some data processing on this dataset
train_M_dataset["action_index"] = train_M_dataset.index // 120
train_M_dataset

#Pred edit
train_M_dataset["real_action"] = 0

jab_ind = [1, 5, 7, 9, 11, 31, 33, 35, 37, 39, 61, 63, 65, 67, 69,
           93, 95, 97, 99, 103, 131, 133, 135, 137, 139, 161, 163, 165, 167, 169,
           191, 193, 195, 197, 199, 223, 225, 227, 229, 231, 253, 255, 257, 259, 261,
           283, 285, 287, 289, 291, 313, 315, 317, 319, 321, 345, 347, 349, 351, 353,
           377, 381, 383, 385, 387, 389, 415, 417, 419, 421, 423, 425, 447, 449, 451, 453, 455,
           481, 483, 485, 487, 489, 513, 515, 517, 519, 521, 549, 551, 553, 555, 557, 559,
           589, 615, 617, 619, 621, 623, 647, 649, 651, 653, 655, 683, 689,
           719, 721, 723, 725, 727, 751, 753, 755, 757, 759, 789, 793, 795, 797, 799, 801, 803, 805, 807]

uppercut_ind = [13, 15, 17, 19, 41, 43, 45, 47, 49, 71, 73, 75, 77, 79,
                107, 111, 113, 115, 117, 141, 143, 145, 147, 149, 171, 173, 175, 177, 179,
                201, 203, 205, 207, 209, 233, 235, 237, 239, 241, 263, 265, 267, 269, 271,
                293, 295, 297, 299, 301, 323, 325, 327, 329, 331, 355, 357, 359, 361, 363,
                391, 393, 395, 397, 399, 427, 429, 431, 433, 457, 459, 461, 465, 467,
                491, 493, 495, 497, 499, 523, 525, 527, 529, 531, 561, 563, 571,
                593, 595, 597, 599, 601, 625, 627, 629, 631, 633, 657, 659, 661, 663, 665,
                693, 695, 697, 699, 701, 703, 729, 731, 733, 735, 737, 761, 765, 767, 771, 773]

hook_ind = [21, 23, 25, 27, 29, 51, 53, 55, 57, 59, 81, 83, 85, 87, 89, 91,
            119, 121, 123, 125, 127, 151, 153, 155, 157, 159, 181, 183, 185, 187, 189,
            211, 213, 215, 217, 219, 221, 243, 245, 247, 249, 251, 273, 275, 277, 279, 281,
            303, 305, 307, 309, 311, 333, 335, 339, 341, 343, 365, 367, 369, 371, 373,
            401, 403, 407, 409, 435, 437, 439, 441, 443, 469, 471, 473, 475, 477,
            501, 503, 505, 507, 509, 535, 537, 539, 541, 543, 545, 579, 581, 583,
            603, 605, 607, 609, 611, 635, 639, 641, 667, 669, 671, 673, 675, 677,
            705, 707, 709, 711, 713, 739, 741, 743, 745, 747, 775, 777, 781, 783, 785]

train_M_dataset.loc[train_M_dataset["action_index"].isin(jab_ind), "real_action"] = 1
train_M_dataset.loc[train_M_dataset["action_index"].isin(uppercut_ind), "real_action"] = 2
train_M_dataset.loc[train_M_dataset["action_index"].isin(hook_ind), "real_action"] = 3



#Now let's do some dataprocessing (rolling average)
data_train_M_filt = train_M_dataset.copy()

for i in ["X", "Y", "Z", "R", "Theta", "Phi"]:
    data_train_M_filt[i] = data_train_M_filt[[i]].rolling(5).mean()
    data_train_M_filt.loc[range(4), [i]] = data_train_M_filt.loc[range(4), [i]]

data_train_M_filt


#Now let's do some more processing


,Time,X,Y,Z,R,Theta,Phi,file_ind,action_index,real_action
0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
1,0.002492,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
2,0.004984,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
3,0.007475,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
4,0.009967,2.601273,9.646319,-0.617728,10.011068,93.536240,74.908119,0,0,0
...,...,...,...,...,...,...,...,...,...,...
416015,24.509064,0.030902,1.152355,9.273292,9.344959,7.096107,88.570166,24,3466,0
416016,24.511562,0.082645,1.192120,9.288144,9.365162,7.341780,86.206181,24,3466,0
416017,24.514059,0.126722,1.229011,9.291019,9.373410,7.584744,84.312027,24,3466,0
416018,24.516556,0.166967,1.272609,9.274250,9.363233,7.890777,82.676460,24,3466,0


In [9]:
# A function that processes all files from a folder
# until the markup.
# The parameters (wind_size, action_max, action_min) for each folder
# were selected in the last paragraph.
def raw_data_to_ready_markup_data(data_path, wind_size, action_max, action_min):
  data_train = pd.DataFrame()
  i = 0
  dir = os.listdir(data_path)
  dir.sort()
  for file_name in dir:
      data_i = pd.read_csv(data_path + file_name, skiprows=skip_fun)
      data_i["file_ind"] = i
      data_train = pd.concat([data_train, data_i], ignore_index=True)
      i += 1
  data_train.rename(columns={"Time (s)": "Time",
                      " X (m/s2)": "X",
                      " Y (m/s2)": "Y",
                      " Z (m/s2)": "Z",
                      " R (m/s2)": "R",
                      " Theta (deg)": "Theta",
                      " Phi (deg)": "Phi",}, inplace=True)

  data_train.sort_values(by=["file_ind", "Time"], inplace=True)

  data_train_filt = data_train.copy()

  for i in ["X", "Y", "Z", "R", "Theta", "Phi"]:
      data_train_filt[i] = data_train_filt[[i]].rolling(5).mean()
      data_train_filt.loc[range(4), [i]] = data_train.loc[range(4), [i]]

  data_train_filt["win_ind"] = data_train_filt.index // wind_size

  data_win = data_train_filt.loc[:, ["R", "win_ind"]]\
      .groupby("win_ind")\
      .aggregate(lambda x: int((x.max() > action_max) or (x.min() < action_min)))\
      .rename(columns={"R": "Active"})\
      .reset_index()

  data_train_filt = pd.merge(data_train_filt, data_win, how='outer', on='win_ind')

  data_train_filt["action_index"] = 0
  action_ind = 0
  for i in range(1, data_train_filt.shape[0]):
      if data_train_filt.loc[i, "Active"] != data_train_filt.loc[i - 1, "Active"]:
          action_ind += 1
      data_train_filt.loc[i, "action_index"] = action_ind

  return(data_train_filt)

In [10]:
data_train_M = raw_data_to_ready_markup_data("data/train_file/train_M/",
                                             100, 13.5, 7)

In [13]:
data_train_M["real_action"] = 0

jab_ind = [1, 5, 7, 9, 11, 31, 33, 35, 37, 39, 61, 63, 65, 67, 69,
           93, 95, 97, 99, 103, 131, 133, 135, 137, 139, 161, 163, 165, 167, 169,
           191, 193, 195, 197, 199, 223, 225, 227, 229, 231, 253, 255, 257, 259, 261,
           283, 285, 287, 289, 291, 313, 315, 317, 319, 321, 345, 347, 349, 351, 353,
           377, 381, 383, 385, 387, 389, 415, 417, 419, 421, 423, 425, 447, 449, 451, 453, 455,
           481, 483, 485, 487, 489, 513, 515, 517, 519, 521, 549, 551, 553, 555, 557, 559,
           589, 615, 617, 619, 621, 623, 647, 649, 651, 653, 655, 683, 689,
           719, 721, 723, 725, 727, 751, 753, 755, 757, 759, 789, 793, 795, 797, 799, 801, 803, 805, 807]

uppercut_ind = [13, 15, 17, 19, 41, 43, 45, 47, 49, 71, 73, 75, 77, 79,
                107, 111, 113, 115, 117, 141, 143, 145, 147, 149, 171, 173, 175, 177, 179,
                201, 203, 205, 207, 209, 233, 235, 237, 239, 241, 263, 265, 267, 269, 271,
                293, 295, 297, 299, 301, 323, 325, 327, 329, 331, 355, 357, 359, 361, 363,
                391, 393, 395, 397, 399, 427, 429, 431, 433, 457, 459, 461, 465, 467,
                491, 493, 495, 497, 499, 523, 525, 527, 529, 531, 561, 563, 571,
                593, 595, 597, 599, 601, 625, 627, 629, 631, 633, 657, 659, 661, 663, 665,
                693, 695, 697, 699, 701, 703, 729, 731, 733, 735, 737, 761, 765, 767, 771, 773]

hook_ind = [21, 23, 25, 27, 29, 51, 53, 55, 57, 59, 81, 83, 85, 87, 89, 91,
            119, 121, 123, 125, 127, 151, 153, 155, 157, 159, 181, 183, 185, 187, 189,
            211, 213, 215, 217, 219, 221, 243, 245, 247, 249, 251, 273, 275, 277, 279, 281,
            303, 305, 307, 309, 311, 333, 335, 339, 341, 343, 365, 367, 369, 371, 373,
            401, 403, 407, 409, 435, 437, 439, 441, 443, 469, 471, 473, 475, 477,
            501, 503, 505, 507, 509, 535, 537, 539, 541, 543, 545, 579, 581, 583,
            603, 605, 607, 609, 611, 635, 639, 641, 667, 669, 671, 673, 675, 677,
            705, 707, 709, 711, 713, 739, 741, 743, 745, 747, 775, 777, 781, 783, 785]

data_train_M.loc[data_train_M["action_index"].isin(jab_ind), "real_action"] = 1
data_train_M.loc[data_train_M["action_index"].isin(uppercut_ind), "real_action"] = 2
data_train_M.loc[data_train_M["action_index"].isin(hook_ind), "real_action"] = 3

In [14]:
data_train_M

,Time,X,Y,Z,R,Theta,Phi,file_ind,win_ind,Active,action_index,real_action
0,0.000000,2.626666,9.605117,-0.391113,9.965471,92.249260,74.705528,0,0,0,0,0
1,0.002492,2.629061,9.648235,-0.525262,10.013806,93.006760,74.757462,0,0,0,0,0
2,0.004984,2.607502,9.645840,-0.642641,10.012706,93.679916,74.873123,0,0,0,0,0
3,0.007475,2.571569,9.676982,-0.731274,10.039508,94.177109,75.118118,0,0,0,0,0
4,0.009967,2.601273,9.646319,-0.617728,10.011068,93.536240,74.908119,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
416015,24.509064,0.030902,1.152355,9.273292,9.344959,7.096107,88.570166,24,4160,0,814,0
416016,24.511562,0.082645,1.192120,9.288144,9.365162,7.341780,86.206181,24,4160,0,814,0
416017,24.514059,0.126722,1.229011,9.291019,9.373410,7.584744,84.312027,24,4160,0,814,0
416018,24.516556,0.166967,1.272609,9.274250,9.363233,7.890777,82.676460,24,4160,0,814,0


In [15]:
#Now let's get some of the vectors we need!
data_train_M_full = data_train_M[data_train_M["real_action"] != 0]

In [16]:
#Let's visualize our dataset!
data_train_M_full

,Time,X,Y,Z,R,Theta,Phi,file_ind,win_ind,Active,action_index,real_action
1000,2.491813,2.380888,9.331071,-0.192287,9.632059,91.143626,75.685904,0,10,1,1,1
1001,2.494305,2.369389,9.307116,-0.172165,9.605727,91.025777,75.717266,0,10,1,1,1
1002,2.496797,2.373222,9.277891,-0.139107,9.577820,90.830576,75.651718,0,10,1,1,1
1003,2.499288,2.386158,9.258248,-0.108924,9.561533,90.651591,75.547499,0,10,1,1,1
1004,2.501780,2.390949,9.229981,-0.085927,9.535121,90.515932,75.477109,0,10,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
413095,17.216600,-4.079297,9.743097,-0.572693,10.587740,93.097337,112.704559,24,4130,1,807,1
413096,17.219095,-4.372506,9.664525,-0.667075,10.638360,93.590367,114.332085,24,4130,1,807,1
413097,17.221592,-4.648468,9.573017,-0.797869,10.681412,94.277255,115.890176,24,4130,1,807,1
413098,17.224091,-4.884185,9.483425,-0.979928,10.721260,95.238635,117.243936,24,4130,1,807,1


In [17]:
#Here are some functions which we will use for calculating some things!
# A function that counts the number of signal crossings of its average value
def n_cross(x):
    x_del = x - x.mean()
    return (np.diff(np.sign(x_del)) != 0).sum()

# A function that counts the number of points
# where the signal is greater than the maximum value of the sensor
def n_sensor_limit_max(x):
    return (x >= 77).sum()

# A function that counts the number of points
# where the signal is greater than the minimum value of the sensor
def n_sensor_limit_min(x):
    return (x <= -77).sum()

In [18]:
# For each action, we will calculate the statistical features that will be used in the model
# The set of statistical features is slightly different for data from the gyroscope and accelerometer
# So we count them separately
data_feature_train_acc = data_train_M_full.loc[:, ["X", "Y", "Z", "R", "action_index"]]\
    .groupby("action_index").aggregate([
    lambda x: x.mean(),
    lambda x: x.std(),
    lambda x: x.max(),
    lambda x: x.min(),
    lambda x: x.max() - x.min(),
    lambda x: np.percentile(x, 25),
    lambda x: np.percentile(x, 50),
    lambda x: np.percentile(x, 75),
    lambda x: moment(x, moment=3),
    lambda x: moment(x, moment=4),
    lambda x: skew(x),
    lambda x: kurtosis(x),
    lambda x: n_cross(x),
    lambda x: n_sensor_limit_max(x),
    lambda x: n_sensor_limit_min(x)])

# Remove transformation artifacts
data_feature_train_acc = data_feature_train_acc.T.reset_index().T.drop(["level_0", "level_1"])
data_feature_train_acc = data_feature_train_acc.apply(lambda x: x.astype(float))

In [19]:
#Let's visualize the set of features here, and let's consider how to train our

data_feature_train_acc

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
action_index,,,,,,,,,,,,,,,,,,,,,
1,-3.436303,16.450873,26.917036,-30.386678,57.303714,-17.535539,2.099896,6.240876,-68.796448,1.415320e+05,...,10.357474,16.434739,27.793253,298.579845,1.183569e+04,0.397121,-1.268816,6.0,0.0,0.0
5,0.407428,16.522916,29.930575,-44.486111,74.416686,-4.087322,4.988629,9.095834,-4394.473349,2.664581e+05,...,10.409625,12.426764,20.746344,4236.592176,1.851868e+05,2.106737,4.295721,6.0,0.0,0.0
7,0.509840,18.114681,33.148690,-54.735498,87.884188,-1.399930,4.243149,9.454080,-6223.081022,4.354265e+05,...,10.550516,13.261916,24.896675,10242.195120,6.074414e+05,2.305662,5.318425,6.0,10.0,0.0
9,-0.903593,18.467763,26.685152,-55.665430,82.350582,-10.223036,3.419816,7.894011,-5303.969927,3.805866e+05,...,10.619172,14.294199,26.987145,7174.705232,4.042368e+05,2.084744,4.779780,6.0,6.0,0.0
11,-0.634991,19.913370,34.095871,-67.340140,101.436010,-7.651107,4.569177,8.866105,-7717.375809,6.603252e+05,...,11.649554,15.844981,26.805397,10671.857461,6.394920e+05,2.088096,4.264110,8.0,10.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799,-0.824770,26.453324,77.694450,-79.292244,156.986694,-9.469651,-3.541986,1.770874,12800.939985,2.686953e+06,...,10.561590,18.906818,35.162386,59689.801534,5.614049e+06,1.882181,2.592737,6.0,51.0,0.0
801,-1.227022,21.290857,77.694450,-78.896030,156.590480,-7.505461,-3.397537,-0.240268,13028.986892,1.672073e+06,...,9.642863,15.292998,26.854634,47041.874068,4.450852e+06,2.405572,5.448019,8.0,45.0,0.0
803,0.846707,27.815781,77.694450,-79.245293,156.939743,-11.662731,-4.483417,-1.458380,19017.223734,2.758352e+06,...,12.446675,25.222106,43.333124,52925.361189,4.990672e+06,1.657148,1.925180,8.0,47.0,0.0


In [20]:
# Similarly for the gyroscope
data_feature_train_cor = data_train_M_full.loc[:, ["Theta", "Phi", "action_index"]]\
    .groupby("action_index").aggregate([
    lambda x: x.mean(),
    lambda x: x.std(),
    lambda x: x.max(),
    lambda x: x.min(),
    lambda x: x.max() - x.min(),
    lambda x: np.percentile(x, 25),
    lambda x: np.percentile(x, 50),
    lambda x: np.percentile(x, 75),
    lambda x: skew(x),
    lambda x: kurtosis(x),
    lambda x: n_cross(x),
    lambda x: n_sensor_limit_max(x),
    lambda x: n_sensor_limit_min(x)])

data_feature_train_cor = data_feature_train_cor.T.reset_index().T.drop(["level_0", "level_1"])
data_feature_train_cor = data_feature_train_cor.apply(lambda x: x.astype(float))

In [21]:
#visualize the data!

data_feature_train_cor

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
action_index,,,,,,,,,,,,,,,,,,,,,
1,112.467093,19.379471,158.210156,74.434425,83.775731,95.361655,113.976215,125.250029,0.356328,-0.595912,...,0.648933,357.504210,61.280922,165.810187,209.609644,0.202002,-0.858733,1.0,170.0,0.0
5,115.398210,22.346805,176.873761,26.002074,150.871687,105.995000,116.344188,128.194099,-0.811674,3.427620,...,2.301244,355.234974,30.037503,77.476532,212.142430,0.732402,-0.704139,2.0,253.0,0.0
7,111.377698,23.547781,165.813235,22.928915,142.884321,98.253705,112.818928,123.993420,-0.670620,2.072761,...,0.598314,357.785615,31.482438,68.446529,189.815917,0.901923,-0.285919,4.0,189.0,0.0
9,108.565692,21.793079,150.961673,19.455994,131.505679,101.811738,109.548494,120.952595,-1.401359,3.728371,...,1.888570,355.379289,44.840714,70.310721,167.262396,0.912986,-0.021068,2.0,186.0,0.0
11,111.386292,30.435741,175.405377,10.508751,164.896626,93.865447,116.086719,129.760573,-0.754631,1.191735,...,1.414889,356.007358,25.728857,64.416328,172.519393,0.718639,-0.589531,6.0,188.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799,110.140538,26.767878,157.054718,26.423527,130.631191,96.223692,115.757401,130.457606,-0.790286,0.055059,...,5.559498,344.381896,102.499425,139.104338,192.989357,0.826625,0.050885,6.0,449.0,0.0
801,107.052639,25.913476,155.279205,38.109283,117.169922,92.964988,109.869065,125.237277,-0.395036,-0.310262,...,7.316483,348.991122,98.682478,130.699410,176.108959,1.050022,0.803359,11.0,642.0,0.0
803,110.260067,25.337122,161.205566,40.700466,120.505101,97.091167,110.736956,127.107185,-0.363580,0.190674,...,7.733699,349.022020,117.926233,145.617657,235.883677,0.573735,-0.530934,6.0,382.0,0.0


In [22]:
# Merge dataset => We received a ready train dataset for the first model
data_feature_train_acc.reset_index(inplace=True)
data_feature_train_cor.reset_index(inplace=True)
X_train = pd.merge(data_feature_train_acc, data_feature_train_cor, on="action_index")
X_train.drop("action_index", axis=1, inplace=True)

In [23]:
X_train

,0_x,1_x,2_x,3_x,4_x,5_x,6_x,7_x,8_x,9_x,...,16_y,17_y,18_y,19_y,20_y,21_y,22_y,23_y,24_y,25_y
0,-3.436303,16.450873,26.917036,-30.386678,57.303714,-17.535539,2.099896,6.240876,-68.796448,1.415320e+05,...,0.648933,357.504210,61.280922,165.810187,209.609644,0.202002,-0.858733,1.0,170.0,0.0
1,0.407428,16.522916,29.930575,-44.486111,74.416686,-4.087322,4.988629,9.095834,-4394.473349,2.664581e+05,...,2.301244,355.234974,30.037503,77.476532,212.142430,0.732402,-0.704139,2.0,253.0,0.0
2,0.509840,18.114681,33.148690,-54.735498,87.884188,-1.399930,4.243149,9.454080,-6223.081022,4.354265e+05,...,0.598314,357.785615,31.482438,68.446529,189.815917,0.901923,-0.285919,4.0,189.0,0.0
3,-0.903593,18.467763,26.685152,-55.665430,82.350582,-10.223036,3.419816,7.894011,-5303.969927,3.805866e+05,...,1.888570,355.379289,44.840714,70.310721,167.262396,0.912986,-0.021068,2.0,186.0,0.0
4,-0.634991,19.913370,34.095871,-67.340140,101.436010,-7.651107,4.569177,8.866105,-7717.375809,6.603252e+05,...,1.414889,356.007358,25.728857,64.416328,172.519393,0.718639,-0.589531,6.0,188.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356,-0.824770,26.453324,77.694450,-79.292244,156.986694,-9.469651,-3.541986,1.770874,12800.939985,2.686953e+06,...,5.559498,344.381896,102.499425,139.104338,192.989357,0.826625,0.050885,6.0,449.0,0.0
357,-1.227022,21.290857,77.694450,-78.896030,156.590480,-7.505461,-3.397537,-0.240268,13028.986892,1.672073e+06,...,7.316483,348.991122,98.682478,130.699410,176.108959,1.050022,0.803359,11.0,642.0,0.0
358,0.846707,27.815781,77.694450,-79.245293,156.939743,-11.662731,-4.483417,-1.458380,19017.223734,2.758352e+06,...,7.733699,349.022020,117.926233,145.617657,235.883677,0.573735,-0.530934,6.0,382.0,0.0
359,0.726817,29.742408,77.694450,-79.256313,156.950763,-13.449894,-5.835438,6.382211,20692.579592,3.189980e+06,...,5.990778,349.492413,122.340234,169.849548,226.768921,0.253039,-0.374830,8.0,361.0,0.0


In [24]:
y_train = data_train_M_full.loc[:, ["real_action", "action_index"]]\
             .groupby("action_index").aggregate(lambda x: list(set(x))[0]).real_action

In [25]:
#Visualize the y_train
y_train

action_index
1      1
5      1
7      1
9      1
11     1
      ..
799    1
801    1
803    1
805    1
807    1
Name: real_action, Length: 361, dtype: int64

In [29]:
#Now let's try the CatBoost algorithm
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=13)
cat_score_train = []
cat_score_test = []

In [ ]:
for n_tree in [250, 275, 300, 325, 350, 400]:
    ctb = CatBoostClassifier(random_seed=17, iterations=n_tree, auto_class_weights="Balanced")
    cv_scores_train = []
    cv_scores_test = []
    for train_idx, test_idx in skf.split(X_train, y_train):
        cv_train_df, cv_valid_df = (X_train.iloc[train_idx, :],
                                    X_train.iloc[test_idx, :])
        y_cv_train, y_cv_valid = y_train.iloc[train_idx], y_train.iloc[test_idx]

        ctb.fit(cv_train_df, y_cv_train)

        cv_scores_train.append(roc_auc_score(y_cv_train, ctb.predict_proba(cv_train_df), multi_class="ovr"))
        cv_scores_test.append(roc_auc_score(y_cv_valid, ctb.predict_proba(cv_valid_df), multi_class="ovr"))
    cat_score_train.append(np.mean(cv_scores_train))
    cat_score_test.append(np.mean(cv_scores_test))

Learning rate set to 0.239603
0:	learn: 0.9280586	total: 226ms	remaining: 56.3s
1:	learn: 0.7912423	total: 265ms	remaining: 32.8s
2:	learn: 0.6835258	total: 305ms	remaining: 25.1s
3:	learn: 0.5909087	total: 348ms	remaining: 21.4s
4:	learn: 0.5259617	total: 387ms	remaining: 19s
5:	learn: 0.4669883	total: 445ms	remaining: 18.1s
6:	learn: 0.4074805	total: 507ms	remaining: 17.6s
7:	learn: 0.3694709	total: 557ms	remaining: 16.8s
8:	learn: 0.3490778	total: 603ms	remaining: 16.1s
9:	learn: 0.3218556	total: 645ms	remaining: 15.5s
10:	learn: 0.2938662	total: 694ms	remaining: 15.1s
11:	learn: 0.2760391	total: 743ms	remaining: 14.7s
12:	learn: 0.2578161	total: 800ms	remaining: 14.6s
13:	learn: 0.2455160	total: 841ms	remaining: 14.2s
14:	learn: 0.2280737	total: 885ms	remaining: 13.9s
15:	learn: 0.2179347	total: 933ms	remaining: 13.7s
16:	learn: 0.2019350	total: 975ms	remaining: 13.4s
17:	learn: 0.1922735	total: 1.01s	remaining: 13.1s
18:	learn: 0.1818097	total: 1.05s	remaining: 12.8s
19:	learn: 0.

In [ ]:
cat_score_train
cat_score_test

[0.9975610635378626,
 0.9968705464429579,
 0.9983178333081615,
 0.9969751103108528,
 0.9968565002293446,
 0.9977952634157784]

In [32]:
#Now let's try support vector machines
clf = sklearn.svm.SVC()
type(X_train)

pandas.core.frame.DataFrame

In [37]:
X_train = X_train.to_numpy()
y_train = y_train.to_numpy()

AttributeError: ignored

In [38]:
y_train

array([1, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 2, 2, 2,
       2, 2, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3,
       3, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 2,
       2, 2, 2, 2, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3,
       3, 3, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 1, 1, 1, 1,
       1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3,
       3, 3, 3, 3, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 1, 1, 1,
       1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2,
       3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 1, 1,
       1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 2, 2, 2, 2,
       2, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 1,
       1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 2,
       2, 2, 3, 3, 3, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 2,
       2, 2, 2, 2, 3, 3, 3, 1, 1, 1, 1, 1, 2, 2, 2,

In [39]:
X_train

array([[ -3.43630296,  16.45087252,  26.917036  , ...,   1.        ,
        170.        ,   0.        ],
       [  0.40742785,  16.5229157 ,  29.9305748 , ...,   2.        ,
        253.        ,   0.        ],
       [  0.50984028,  18.11468093,  33.14869   , ...,   4.        ,
        189.        ,   0.        ],
       ...,
       [  0.84670656,  27.81578117,  77.69445   , ...,   6.        ,
        382.        ,   0.        ],
       [  0.7268166 ,  29.74240767,  77.69445   , ...,   8.        ,
        361.        ,   0.        ],
       [ -1.01243979,  28.4264237 ,  77.69445   , ...,   6.        ,
        384.        ,   0.        ]])

In [40]:
y_train.shape

(361,)

In [41]:
np.reshape(y_train, (361, 1))

array([[1],
       [1],
       [1],
       [1],
       [1],
       [2],
       [2],
       [2],
       [2],
       [3],
       [3],
       [3],
       [3],
       [3],
       [1],
       [1],
       [1],
       [1],
       [1],
       [2],
       [2],
       [2],
       [2],
       [2],
       [3],
       [3],
       [3],
       [3],
       [3],
       [1],
       [1],
       [1],
       [1],
       [1],
       [2],
       [2],
       [2],
       [2],
       [2],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [1],
       [1],
       [1],
       [1],
       [1],
       [2],
       [2],
       [2],
       [2],
       [2],
       [3],
       [3],
       [3],
       [3],
       [3],
       [1],
       [1],
       [1],
       [1],
       [1],
       [2],
       [2],
       [2],
       [2],
       [2],
       [3],
       [3],
       [3],
       [3],
       [3],
       [1],
       [1],
       [1],
       [1],
       [1],
       [2],
       [2],
       [2],
    

In [42]:
X_train
X_train.shape
y_train

array([1, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 2, 2, 2,
       2, 2, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3,
       3, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 2,
       2, 2, 2, 2, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3,
       3, 3, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 1, 1, 1, 1,
       1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3,
       3, 3, 3, 3, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 1, 1, 1,
       1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2,
       3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 1, 1,
       1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 2, 2, 2, 2,
       2, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 1,
       1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 2,
       2, 2, 3, 3, 3, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 2,
       2, 2, 2, 2, 3, 3, 3, 1, 1, 1, 1, 1, 2, 2, 2,

In [43]:
X_train.shape

(361, 86)

In [44]:
clf.fit(X_train, y_train)

SVC()

In [45]:
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.preprocessing import LabelBinarizer

In [46]:
classif = OneVsRestClassifier(estimator=SVC(random_state=0))
help(OneVsRestClassifier)

Help on class OneVsRestClassifier in module sklearn.multiclass:

class OneVsRestClassifier(sklearn.base.MultiOutputMixin, sklearn.base.ClassifierMixin, sklearn.base.MetaEstimatorMixin, sklearn.base.BaseEstimator)
 |  OneVsRestClassifier(estimator, *, n_jobs=None, verbose=0)
 |  
 |  One-vs-the-rest (OvR) multiclass strategy.
 |  
 |  Also known as one-vs-all, this strategy consists in fitting one classifier
 |  per class. For each classifier, the class is fitted against all the other
 |  classes. In addition to its computational efficiency (only `n_classes`
 |  classifiers are needed), one advantage of this approach is its
 |  interpretability. Since each class is represented by one and one classifier
 |  only, it is possible to gain knowledge about the class by inspecting its
 |  corresponding classifier. This is the most commonly used strategy for
 |  multiclass classification and is a fair default choice.
 |  
 |  OneVsRestClassifier can also be used for multilabel classification. T

In [47]:
predictions = classif.fit(X_train, y_train).predict(X_train)
predictions

array([2, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 2, 2, 2,
       2, 3, 1, 3, 3, 3, 1, 1, 1, 1, 1, 1, 3, 3, 2, 2, 2, 1, 1, 3, 3, 3,
       2, 1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 3,
       3, 2, 3, 2, 3, 3, 2, 3, 3, 1, 1, 1, 1, 1, 3, 2, 2, 2, 3, 2, 1, 2,
       3, 3, 1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 1, 1, 1,
       1, 3, 2, 2, 2, 2, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3,
       3, 1, 3, 1, 1, 1, 1, 1, 1, 3, 3, 1, 1, 1, 3, 1, 3, 1, 3, 1, 1, 1,
       1, 1, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 1, 1, 1, 1, 1, 3, 3, 3, 3, 3,
       1, 3, 1, 3, 3, 1, 3, 1, 3, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 1, 1,
       1, 1, 2, 1, 2, 1, 2, 2, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 2, 2, 1, 2,
       2, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 2, 3, 3, 3, 1,
       1, 2, 1, 1, 3, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 1, 2, 1, 1, 2, 1, 2,
       2, 2, 3, 3, 3, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 1, 2, 2, 1, 2, 3,
       2, 2, 1, 2, 3, 3, 3, 1, 1, 1, 1, 1, 2, 3, 2,

In [48]:
#here's how we get some accuracy
sklearn.metrics.accuracy_score(y_train, predictions)

0.7506925207756233

In [49]:
#What does C = 100.00, hyperparameter tuning this!

classif = OneVsOneClassifier(estimator=SVC(random_state=0, kernel = 'linear', C = 0.1))
predictions = classif.fit(X_train, y_train).predict(X_train)
predictions
sklearn.metrics.accuracy_score(y_train, predictions)


0.778393351800554